In [1]:
import os
import sys

MULTIPOLY_FOLDER = os.path.dirname(os.path.dirname(os.getcwd()))
POLYFFUSION_CKPT_PATH = os.path.join(MULTIPOLY_FOLDER, r"polyffusion_ckpts/ldm_chd8bar/sdf+pop909wm_mix16_chd8bar/01-11_102022/chkpts/weights_best.pt")
CHORD_CKPT_PATH = os.path.join(MULTIPOLY_FOLDER, r"pretrained/chd8bar/weights.pt")

sys.path.append(MULTIPOLY_FOLDER)

from src.models.prepare_model import get_diffusion
multipoly_unet_params_dict = {}
multipoly_unet_params_dict["n_intertrack_head"] = 4
multipoly_unet_params_dict["num_intertrack_encoder_layers"] = 2
multipoly_unet_params_dict["intertrack_attention_levels"] = [3]
diffusion = get_diffusion(POLYFFUSION_CKPT_PATH, CHORD_CKPT_PATH, multipoly_unet_params_dict)

print(diffusion.eps_model.get_norm_of_trainable_params())

---------------loading polyffusion weights-------------------
input_blocks.10.2.attention.layers.0.self_attn.in_proj_weight
input_blocks.10.2.attention.layers.0.self_attn.in_proj_bias
input_blocks.10.2.attention.layers.0.self_attn.out_proj.weight
input_blocks.10.2.attention.layers.0.self_attn.out_proj.bias
input_blocks.10.2.attention.layers.0.linear1.weight
input_blocks.10.2.attention.layers.0.linear1.bias
input_blocks.10.2.attention.layers.0.linear2.weight
input_blocks.10.2.attention.layers.0.linear2.bias
input_blocks.10.2.attention.layers.0.norm1.weight
input_blocks.10.2.attention.layers.0.norm1.bias
input_blocks.10.2.attention.layers.0.norm2.weight
input_blocks.10.2.attention.layers.0.norm2.bias
input_blocks.10.2.attention.layers.1.self_attn.in_proj_weight
input_blocks.10.2.attention.layers.1.self_attn.in_proj_bias
input_blocks.10.2.attention.layers.1.self_attn.out_proj.weight
input_blocks.10.2.attention.layers.1.self_attn.out_proj.bias
input_blocks.10.2.attention.layers.1.linear1.w

In [2]:
from src.data.dataloader import get_train_val_dataloaders
train_dl, val_dl = get_train_val_dataloaders(
    data_folder="/root/autodl-tmp/multipoly/data/lmd/lpd_5_midi/",
    batch_size=7,
    num_workers=4,
    pin_memory=True
)


/root/miniconda3/envs/poly/lib/python3.10/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


KeyboardInterrupt: 

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
diffusion = diffusion.to(device)

In [ ]:
for x in train_dl:
    pr,chord = x
    
    pr = pr.to(device)
    chord = chord.to(device)

    loss = diffusion.loss(pr, chord)
    print(loss)

    break

tensor(0.0012, device='cuda:0', grad_fn=<MseLossBackward0>)


In [ ]:
# chord_single = chord[4]
# chord_single = chord_single.reshape(1,32,36)
# print(chord_single.shape)

In [ ]:
bs = 1
track_num = 4
import torch
uncond_cond = torch.randn(1,1,512)



In [ ]:
diffusion = diffusion.to(device)
uncond_cond = uncond_cond.to(device)
# chord_single = chord_single.to(device)


In [ ]:


#gen_cond = diffusion.sample([1,4,2,128,128],chord_single,uncond_scale=5.0, uncond_cond=uncond_cond)

In [ ]:
gen_uncond = diffusion.sample([1,4,2,128,128],uncond_cond=uncond_cond,uncond_scale=0.0)

100%|██████████| 1000/1000 [00:22<00:00, 44.77it/s]


In [ ]:
print(gen_uncond.shape)

torch.Size([1, 4, 2, 128, 128])


In [ ]:
import pretty_midi as pm

def custom_round(x):
    if x > 0.95 and x < 1.05:
        return 1
    else:
        return 0

def prmat2c_to_midi_file(
    prmat2c, fpath, is_custom_round=True
):
    print(f"prmat2c : {prmat2c.shape}")
    midi = pm.PrettyMIDI()
    piano_program = pm.instrument_name_to_program("Acoustic Grand Piano")
    origin = pm.Instrument(program=piano_program)
    t = 0
    n_step = prmat2c.shape[2]
    t_bar = int(n_step / 8)
    for bar_ind, bars in enumerate(prmat2c):
        onset = bars[0]
        sustain = bars[1]
        for step_ind, step in enumerate(onset):
            for key, on in enumerate(step):
                if is_custom_round:
                    on = int(custom_round(on))
                else:
                    on = int(round(on))
                if on > 0:
                    dur = 1
                    while step_ind + dur < n_step:
                        if not (int(round(sustain[step_ind + dur, key])) > 0):
                            break
                        dur += 1
                    note = pm.Note(
                        velocity=80,
                        pitch=key,
                        start=t + step_ind * 1 / 8,
                        end=min(t + (step_ind + dur) * 1 / 8, t + t_bar),
                    )
                    
                    origin.notes.append(note)
        t += t_bar
    midi.instruments.append(origin)
    midi.write(fpath)


import numpy as np
def chd_to_midi_file(chords, output_fpath, one_beat=0.5):
    """
    retrieve midi from chords
    """
    if "Tensor" in str(type(chords)):
        chords = chords.cpu().detach().numpy()
    midi = pm.PrettyMIDI()
    piano_program = pm.instrument_name_to_program("Acoustic Grand Piano")
    piano = pm.Instrument(program=piano_program)
    t = 0.0
    for chord in chords:
        
        if chord.shape[0] == 14:
            root = int(chord[0])
            chroma = chord[1:13].astype(int)
            bass = int(chord[13])
        elif chord.shape[0] == 36:
            root = int(chord[0:12].argmax())
            chroma = chord[12:24].astype(int)
            bass = int(chord[24:].argmax())

        chroma = np.roll(chroma, -bass)
        c3 = 48
        for i, n in enumerate(chroma):
            if n == 1:
                note = pm.Note(
                    velocity=80,
                    pitch=c3 + i + bass,
                    start=t * one_beat,
                    end=(t + 1) * one_beat,
                )
                piano.notes.append(note)
        t += 1

    midi.instruments.append(piano)
    midi.write(output_fpath)



In [ ]:
# chd_to_midi_file(chord_single[0], f"exp/chord.mid")

for i in range(4):
    track_gen = gen_uncond[:,i].cpu().numpy()
    prmat2c_to_midi_file(track_gen, f"exp/diffusion_test_uncond_{i}.mid")

prmat2c : (1, 2, 128, 128)
prmat2c : (1, 2, 128, 128)
prmat2c : (1, 2, 128, 128)
prmat2c : (1, 2, 128, 128)
